In [1]:
import open_clip

import torch
from PIL import Image
import random
import glob
import os
import numpy as np
import time
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [3]:
os.makedirs('./models/', exist_ok=True) 

## Load model

In [10]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')
tokenizer = open_clip.get_tokenizer('ViT-B-32-quickgelu')

In [11]:
def get_image_embedding(image_path):
    image = Image.open(image_path)
    tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(tensor)
    return embedding.flatten().cpu().numpy() 

## prepare dataset

In [12]:
imglist = glob.glob('./data/HistoricalColor-ECCV2012/data/imgs/decade_database/**/*.jpg')
random.shuffle(imglist) 

In [13]:
X = []
labels = []
for img in imglist:
    embedding = get_image_embedding(img)
    #features = np.concatenate((embedding1, embedding2))
    label = int(img.split('/')[6][:-1])
    labels.append(label)
    X.append(embedding)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2)

## Train classifier

In [15]:
clf = LogisticRegression(random_state=0, max_iter=1000, verbose=0)
clf.fit(X_train, y_train)

timestamp = time.strftime("%Y%m%d-%H%M")

y_pred = clf.predict(X_test)

score =accuracy_score(y_test,y_pred)
print(score)

0.5509433962264151


## Saving model

In [135]:
with open(f'./models/{timestamp}.pkl', 'wb') as f:
    pickle.dump(clf, f)
print('model saved!')

model saved!
